# RK DEAP Chapter 1: Travelling Salesman

Learning DEAP from examples, Ronn Kling  
https://www.amazon.com/Learning-DEAP-examples-Evolutionary-evolutionary-ebook/dp/B06XHXD2SF
 
We are using this code as starting point but annotate the code and add additional information.

In [5]:
import matplotlib.pyplot as plt
import sys
import array
import random
import numpy as np

from deap import  algorithms
from deap import  base
from deap import  creator
from deap import  tools

%matplotlib inline


In [13]:
numCities = 10
doVerbose = True



In [ ]:
random.seed(169)
x = np.random.rand(numCities)
y = np.random.rand(numCities)
plt.plot(x,y)
plt.show()


## Creator 

https://deap.readthedocs.io/en/master/tutorials/basic/part1.html#creating-types  
https://deap.readthedocs.io/en/master/tutorials/basic/part1.html#individual  
https://deap.readthedocs.io/en/master/api/creator.html  

The `creator` class factory provides the facility to use the DEAP classes as base classes but then to derive new classes for new problems.

New classes can be built from any imaginable type, from list to set, dict, PrimitiveTree and more

    deap.creator.create(name, base[, attribute[, ...]])
    
The create() function takes at least two arguments, a name for the newly created class and an existing  base class. Any subsequent argument becomes an attribute of the class.

The new class can have attributes defined by the subsequent keyword arguments passed to the function `create`. If the argument is a class (a name without the parenthesis), the `__init__` function of the supplied class type is called in the initialization of an instance of the newly created  object. It seems that any required parameters in the existing class' `__init__` is taken from the rest of the `create()` parameter list.  
The newly created class (using the name supplied) is then added as an attribute of the `creator` class. Otherwise, if the argument is not a class, (for example an int), it is added as a `static` attribute of the `creator` class.



https://deap.readthedocs.io/en/master/tutorials/basic/part1.html#creating-types

The provided `Fitness` class is an abstract class that needs a weights attribute in order to be functional. A minimizing fitness is built using negatives weights, while a maximizing fitness has positive weights. For example, the following line creates, in the `creator`, a ready to use single objective minimizing fitness named `FitnessMin`:

    creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
    
A multi-objective fitness function with have more elements in the weights tuple.  The tuple `weights=(-1.0,1.0)` means that the first objective is minimised and the the second objective is maximised. The weights can also be used to vary the importance of each objective one against another. This means that the weights can be any real number and only the sign is used to determine if a maximization or minimization is done.

The `creator.create` below creates a new `creator` method that will create an instance of the `FitnessMin` class when invoked.

In [21]:
creator.create("FitnessMin", base.Fitness,weights=(-1.0,))


C:\ProgramData\Anaconda3\lib\site-packages\deap-1.2.2-py3.7-win-amd64.egg\deap\creator.py:141: RuntimeWarning: A class named 'FitnessMin' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)


https://deap.readthedocs.io/en/master/tutorials/basic/part1.html#individual  
https://docs.python.org/3/library/array.html      

    class array.array(typecode[, initializer])

Instantiates an efficient array of type defined in typecode.

The `create.Individual` method can create arbitrary data types: lists, dicts, etc. Variations of this type are possible by inheriting from `array.array` or `numpy.ndarray` as following.

    creator.create("Individual", array.array, typecode="d", fitness=creator.FitnessMax)
    creator.create("Individual", numpy.ndarray, fitness=creator.FitnessMax)

Type inheriting from arrays needs a typecode on initialization, just as the original class.

Finally, the `create.Individual` method  must be given a fitness class, created by invoking the creator method that was defined above.


In [35]:
creator.create("Individuals",array.array,typecode='i', fitness=creator.FitnessMin)

## Creating toolbox attributes

https://deap.readthedocs.io/en/master/tutorials/basic/part2.html#operators-and-algorithms  

The toolbox is intended to contain all the evolutionary tools, from the object initializers to the evaluation operator. It allows easy configuration of each algorithm. The toolbox has basically two methods, register() and unregister(), that are used to add or remove tools from the toolbox.

In the TSP problem, each city is represented by an index (a number) in the range `range(0,numCities)`.  The index values are sufficient for this purpose, we do not have to use city names.

Create the attribute `indices` to represent the indexes or indices of the cities.

The `random.sample(population, k)` function returns a `k` length list of unique elements chosen from the `population` sequence or set, used for random sampling without replacement (meaning that items are only used once).

https://pynative.com/python-random-sample/

So, in the code below

1. Register an attribute list with numCities, selected randomly from `range(numCities)`

        tooolbox.register('indices',random.sample,range(numCities),numCities)

2. Register the `individual` toolbox method that will create the individuals (the lists of cities), from the attribute defined just above, in the line

        toolbox.register('individual',tools.initIterate, creator.Individual, toolbox.indices)
        
    The parameter `tools.initIterate` is a built-in DEAP function taking two arguments: the function to be called (`creator.Individual`) and the the argument to be supplied (`toolbox.indices`).  

3. Finally, create the population of individuals each comprising a list of `toolbox.individual` instances, which in turn are lists of city indices.

        toolbox.register('population',tools.initRepeat,list,toolbox.individual)

In [40]:
toolbox = base.Toolbox()
toolbox.register('indices',random.sample,range(numCities),numCities)
toolbox.register('individual',tools.initIterate,creator.Individual, toolbox.indices)
toolbox.register('population',tools.initRepeat,list,toolbox.individual)

In [41]:
if doVerbose:
    ind = toolbox.individual()
    print('Demonstrate the indices, individual and population:')
    print(f'indices (city names): {toolbox.indices()}') 
    print(f'Individual (create sequence of cities): {creator.Individual()}')  
    print(f'individual (one sequence of cities): {toolbox.individual()}')  
    print(f'population (population of sequences of cities): {toolbox.population(n=4)}')  


Demonstrate the indices, individual and population:
indices (city names): [8, 5, 3, 9, 1, 6, 7, 0, 4, 2]
Individual (create sequence of cities): Individual('i')
individual (one sequence of cities): Individual('i', [3, 1, 6, 5, 2, 7, 9, 0, 8, 4])
population (population of sequences of cities): [Individual('i', [2, 6, 4, 0, 3, 5, 9, 8, 1, 7]), Individual('i', [0, 6, 8, 7, 2, 1, 5, 9, 4, 3]), Individual('i', [6, 2, 4, 1, 3, 8, 0, 5, 9, 7]), Individual('i', [7, 1, 5, 4, 3, 8, 6, 0, 9, 2])]


## Crossover / Mating

https://deap.readthedocs.io/en/master/tutorials/basic/part2.html#using-the-toolbox  
https://deap.readthedocs.io/en/master/tutorials/basic/part2.html#crossover  

 There is a variety of crossover operators in the deap.tools module. Each crossover has its own characteristics and may be applied to different types of individuals. Be careful to read the documentation of the selected operator in order to avoid undesirable behaviour.

The general rule for crossover operators is that they only mate individuals, this means that an independent copies must be made prior to mating the individuals if the original individuals have to be kept or are references to other individuals (see the selection operator).

In the travelling salesman problem, each city must be visited and no city must be visited twice. The travel sequence is the `individual` defined above. 
The crossover method must therefore mate the `individuals` in the `population` to comply with the above requirement. DEAP has three different crossover methods that will comply and work with indices:

1. [deap.tools.cxOrdered(ind1, ind2)](https://deap.readthedocs.io/en/master/api/tools.html#deap.tools.cxOrdered)

    Executes an ordered crossover (OX) on the input individuals. The two individuals are modified in place. This crossover *only works with index individual sequences*.  A portion of one parent is mapped to a portion of the other parent. Outside the filled portion, the rest is filled up by the remaining genes, where already present genes are omitted and the order is preserved. See 
    [here](http://www.dmi.unict.it/mpavone/nc-cs/materiale/moscato89.pdf) and [here](https://stackoverflow.com/questions/11782881/how-to-implement-ordered-crossover).
    
2. [ deap.tools.cxUniformPartialyMatched(ind1, ind2, indpb)](https://deap.readthedocs.io/en/master/api/tools.html#deap.tools.cxUniformPartialyMatched)
Executes a uniform partially matched crossover (UPMX) on the input individuals. The two individuals are modified in place. This crossover *only works with index individual sequences*.  In this method, two crossover points are selected at random and PMX proceeds by position-wise exchanges. The two crossover points give matching selection.  It affects crossover by position-by-position exchange operations.

3. [ deap.tools.cxPartialyMatched(ind1, ind2)](https://deap.readthedocs.io/en/master/api/tools.html#deap.tools.cxPartialyMatched) 
Executes a partially matched crossover (PMX) on the input individuals. The two individuals are modified in place. This crossover *only works with index individual sequences*. In this method, two crossover points are selected within a certain range of the parents and PMX proceeds by position-wise exchanges. The two crossover points give matching selection.  It affects crossover by position-by-position exchange operations.


In [42]:
toolbox.register('mate',tools.cxOrdered)


## Mutation

https://deap.readthedocs.io/en/master/tutorials/basic/part2.html#mutation  

There is a variety of mutation operators in the deap.tools module. Each mutation has its own characteristics and may be applied to different types of individuals. Be careful to read the documentation of the selected operator in order to avoid undesirable behaviour.

The general rule for mutation operators is that they only mutate, this means that an independent copy must be made prior to mutating the individual if the original individual has to be kept or is a reference to another individual (see the selection operator).

1. [deap.tools.mutShuffleIndexes(individual, indpb)](https://deap.readthedocs.io/en/master/api/tools.html#deap.tools.mutShuffleIndexes)

    Shuffle the attributes of the input individual and return the mutant. The individual is expected to be a sequence. The `indpb` argument is the probability of each attribute to be moved. Usually this mutation is applied on vector of indices. 
    
The following code mutates with a probability of 0.05 and alias the method `tools.mutShuffleIndexes` to ``mutuate`.
 
 
 

In [43]:
toolbox.register('mutate',tools.mutShuffleIndexes, indpb=0.05)

## Select

https://deap.readthedocs.io/en/master/tutorials/basic/part2.html#selection

Selection is made among a population by the selection operators that are available in the deap.tools module. The selection operator usually takes as first argument an iterable container of individuals and the number of individuals to select. It returns a list containing the references to the selected individuals.

It is very important here to note that the selection operators does not duplicate any individual during the selection process. If an individual is selected twice and one of either object is modified, the other will also be modified. Only a reference to the individual is copied. Just like every other operator it selects and only selects. 

1.  [deap.tools.selTournament(individuals, k, tournsize, fit_attr='fitness')](https://deap.readthedocs.io/en/master/api/tools.html#selection)

    Select the best individual among `tournsize` randomly chosen individuals, `k` times. The list returned contains references to the input individuals.
    
    Tournament is a common selection choice.


In [45]:
toolbox.register('select',tools.selTournament, tournsize=3)

## Evaluation function



Types of crossover: https://deap.readthedocs.io/en/master/api/tools.html#crossover

The order of travel is the chromosome, this is important that the indices must be unique and not missing values. Therefore [xcOrdered](https://deap.readthedocs.io/en/master/api/tools.html#deap.tools.cxOrdered) crossover must be used. The docs states:

     deap.tools.cxOrdered(ind1, ind2)

